In [ ]:
!pip install requests PyMuPDF
!pip install requests

In [ ]:
import fitz  # PyMuPDF
from ollama_api import OllamaApi  # Oder aus deinem eigenen Modul importieren
import json

# === Funktion: Text aus PDF extrahieren ===
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text.strip()

# === Funktion: Nachhaltigkeitsversprechen extrahieren lassen ===
def extract_sustainability_promises(text, model_name="phi4:latest"):
    prompt = (
        "Extrahiere alle Aussagen oder Abschnitte, in denen das Unternehmen Nachhaltigkeit, "
        "Umweltschutz, soziale Verantwortung oder ähnliche Versprechen macht. "
        "Gib nur die relevanten Textstellen als Liste zurück.\n\n"
        f"Text:\n{text}"
    )

    schema = {
        "type": "object",
        "properties": {
            "versprechen": {
                "type": "array",
                "items": {"type": "string"}
            }
        },
        "required": ["versprechen"]
    }

    response = OllamaApi.completion(prompt, model=model_name, schema=schema)
    return response

# === Hauptprogramm ===
if __name__ == "__main__":
    pdf_path = "2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf"  # Pfad zur PDF-Datei

    print("📄 Lese PDF-Datei ein...")
    text = extract_text_from_pdf(pdf_path)

    print("🤖 Sende an LLM zur Analyse...")
    result = extract_sustainability_promises(text)

    print("\n🔍 Extrahierte Nachhaltigkeitsversprechen:")
    if result["result"]:
        for i, versprechen in enumerate(result["result"]["versprechen"], 1):
            print(f"{i}. {versprechen}")
    else:
        print("Keine oder unklare Antwort erhalten.")
        print("Info:", result.get("info"))